In [1]:
import pandas as pd

df = pd.read_csv("train(1).csv")
df["text"][7610]

'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ'

In [2]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
import re 
from num2words import num2words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
def pre_df(text):
    text = text.lower()
    text = re.sub('[@#:;!/$.,%^|?&"\'{}[\]\\\\]' , ' ' , text)
    text = re.sub(' " ' , ' ' , text)
    text = re.sub('[https?://|www\.]+' , ' ' , text)
    text = re.sub('["]' , ' ' , text)
    text = re.sub("[-_]" , ' ' , text)
    text = re.sub("[']" , ' ' , text)
    text = re.sub("<.*?>" , ' ' , text)

    stop_words = set(stopwords.words("english"))
    T_words = word_tokenize(text)


    T_words = []
    for i in T_words:
        if i not in stop_words:
            T_words.append(i)

    stmr = PorterStemmer()
    for word in T_words:
        stmrw = stmr.stem(word)
        T_words.append(stmrw)

    numbers = re.findall("\d+" , text)
    for number in numbers:
        word_by_word = num2words(int(number) , lang="en")
        text = text.replace(number , word_by_word)
    
    return text

In [4]:
sd = pre_df(df["text"][7610])
sd

'mone ninety-four  0one four u c  fivekm   of volcano  a aii        co zd oydeightebj'

In [5]:
df["text"] = df["text"].apply(pre_df)

In [6]:
#TFIDF VECTORIZATION 
#TF = no of rep(good , girl) of word in sen/total no of words in sen 
#TD = log(no of total sen / no of sentence contanining the word(good , girl))
#sen1 = good boy
#sen2 = good girl
#sen3 = good girl boy
#> good = TF = 1/2 , 1/2 , 1/3 , TD = log(3/3)
#> girl = TF = 0 , 1/2 , 1/3 , TD = log(3/2)
#> boy = TF = 1/2 , 0 , 1/3 , TD = log(3/2)
#got itt! NOW TF*IDF 
#no.sen \ good          \          girl \          boy
#sen:1  1/2*log(3/3)=0      0*log(3/2)=0     1/2*log(3/3)
#sen:2 1/2*log(3/3)=0       1/2*log(3/2)     0*log(3/2)=0
#sen:3 1/3*log(3/3)=0       1/3*log(3/2)    1/3*log(3/3)

In [7]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,our deed are e rea on of i ear quake may ...,1
1,4,NaN,NaN,fore fire near la ronge a k canada,1
2,5,NaN,NaN,all re iden a ked o el er in lace are be...,1
3,6,NaN,NaN,thirteen zero eo le receive ildfire evacua...,1
4,7,NaN,NaN,ju go en i o o from ruby ala ka a mo...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,o gian crane olding a bridge colla e in o ...,1
7609,10870,NaN,NaN,aria a rary e a nie e ou of con rol ild...,1
7610,10871,NaN,NaN,mone ninety-four 0one four u c fivekm of v...,1
7611,10872,NaN,NaN,olice inve iga ing af er an e bike collided ...,1


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
encode = TfidfVectorizer()
x1 = encode.fit_transform(df["text"])
df1 = pd.DataFrame(x1.toarray() , columns=encode.get_feature_names_out())
df = pd.concat([df , df1] ,axis=1)

In [9]:
import joblib
vfile = "joblib_v.pkl"
joblib.dump(encode , vfile)

['joblib_v.pkl']

In [10]:
df

,id,keyword,location,text,target,021five,0eight,0five,0four,0one,...,ûïnobody,ûïnumber,ûïric,ûïyou,ûò,ûòeight,ûòåêcnbc,ûó,ûókody,ûónegligence
0,1,NaN,NaN,our deed are e rea on of i ear quake may ...,1,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,NaN,NaN,fore fire near la ronge a k canada,1,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,NaN,NaN,all re iden a ked o el er in lace are be...,1,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,NaN,NaN,thirteen zero eo le receive ildfire evacua...,1,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,NaN,NaN,ju go en i o o from ruby ala ka a mo...,1,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,o gian crane olding a bridge colla e in o ...,1,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7609,10870,NaN,NaN,aria a rary e a nie e ou of con rol ild...,1,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7610,10871,NaN,NaN,mone ninety-four 0one four u c fivekm of v...,1,0.0,0.0,0.0,0.0,0.332801,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7611,10872,NaN,NaN,olice inve iga ing af er an e bike collided ...,1,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
x = df.drop(columns=(["id" , "location" , "keyword" , "target" , "text"]))
x = pd.DataFrame(x)
x

,021five,0eight,0five,0four,0one,0six,0six0,10seven,12four,1390fivefive,...,ûïnobody,ûïnumber,ûïric,ûïyou,ûò,ûòeight,ûòåêcnbc,ûó,ûókody,ûónegligence
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7609,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7610,0.0,0.0,0.0,0.0,0.332801,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7611,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
y = df["target"]
y = pd.DataFrame(y)
y

,target
0,1
1,1
2,1
3,1
4,1
...,...
7608,1
7609,1
7610,1
7611,1


In [13]:
y.value_counts()

target
0         4342
1         3271
Name: count, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
x_tr , x_ts , y_tr , y_ts = train_test_split(x , y , test_size=0.2, random_state=52)
x_tr.shape

(6090, 16125)

In [15]:
print(x_ts.shape , x_tr.shape)
print(y_tr.shape , y_ts.shape)

(1523, 16125) (6090, 16125)
(6090, 1) (1523, 1)


In [16]:
x_tr

,021five,0eight,0five,0four,0one,0six,0six0,10seven,12four,1390fivefive,...,ûïnobody,ûïnumber,ûïric,ûïyou,ûò,ûòeight,ûòåêcnbc,ûó,ûókody,ûónegligence
7174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.ensemble import RandomForestClassifier
r = RandomForestClassifier()

In [18]:
model = r.fit(x_tr , y_tr)

/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future

In [19]:
model

RandomForestClassifier()

In [20]:
xp = model.predict(x_ts)

/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [21]:
print(xp)
print(pd.DataFrame(xp))
print(xp.shape)

[0 0 0 ... 0 0 0]
      0
0     0
1     0
2     0
3     0
4     0
...  ..
1518  0
1519  0
1520  0
1521  0
1522  0

[1523 rows x 1 columns]
(1523,)


In [22]:
print(x_ts)
print(x_ts.shape)

      021five  0eight  0five  0four  0one  0six  0six0  10seven  12four  \
269       0.0     0.0    0.0    0.0   0.0   0.0    0.0      0.0     0.0   
7057      0.0     0.0    0.0    0.0   0.0   0.0    0.0      0.0     0.0   
5161      0.0     0.0    0.0    0.0   0.0   0.0    0.0      0.0     0.0   
1873      0.0     0.0    0.0    0.0   0.0   0.0    0.0      0.0     0.0   
7275      0.0     0.0    0.0    0.0   0.0   0.0    0.0      0.0     0.0   
...       ...     ...    ...    ...   ...   ...    ...      ...     ...   
4425      0.0     0.0    0.0    0.0   0.0   0.0    0.0      0.0     0.0   
1275      0.0     0.0    0.0    0.0   0.0   0.0    0.0      0.0     0.0   
6029      0.0     0.0    0.0    0.0   0.0   0.0    0.0      0.0     0.0   
6798      0.0     0.0    0.0    0.0   0.0   0.0    0.0      0.0     0.0   
7478      0.0     0.0    0.0    0.0   0.0   0.0    0.0      0.0     0.0   

      1390fivefive  ...  ûïnobody  ûïnumber  ûïric  ûïyou   ûò  ûòeight  \
269            0.0  ... 

In [23]:
print(y_ts)
print(y_ts.shape)

      target
269        1
7057       0
5161       0
1873       1
7275       1
...      ...
4425       1
1275       0
6029       0
6798       1
7478       0

[1523 rows x 1 columns]
(1523, 1)


In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(xp , y_ts)

/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/hi/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future

0.804333552199606

In [25]:
import joblib
file= "joblib_model.pkl"
joblib.dump(model , file)

['joblib_model.pkl']